In [1]:
import pandas as pd
import numpy as np
import math
import yaml
from pathlib import Path

pd.set_option("display.max_rows", 9999)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", 999)
pd.set_option("display.precision", 2)

from db_psql_model import DatabaseCursor

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
OPTION_DEV = "-c search_path=dev"
OPTION_PROD = "-c search_path=prod"

MoM FFBL Weekly Rankings Replacement 

In [13]:
"""
MoM FFBL Weekly Rankings Replacement 2pt system
"""
matchups_query = """SELECT * FROM dev.weeklyleaguematchups"""
teams_query = """SELECT team_key, name, nickname, game_id FROM dev.leagueteams WHERE game_id >= 390"""
matchups = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    matchups_query
)
teams = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(teams_query)
matchups = matchups[
    (matchups["is_playoffs"] == 0)
    & (matchups["is_consolation"] == 0)
    & (matchups["game_id"] >= 390)
]
matchups_a = matchups.copy()
matchups_b = matchups.copy()
matchups_b_cols = list(matchups_b.columns)
rename_columns = {}
for col in matchups_b_cols:
    if "team_a" in col:
        rename_columns[col] = f"team_b{col[6:]}"
    elif "team_b" in col:
        rename_columns[col] = f"team_a{col[6:]}"
matchups_b.rename(columns=rename_columns, inplace=True)
matchups = pd.concat([matchups_a, matchups_b])
matchups.sort_values(["week_start", "team_a_team_key"], inplace=True)
matchups.reset_index(drop=True, inplace=True)
matchups.drop(
    [
        "is_matchup_recap_available",
        "is_tied",
        "matchup_recap_title",
        "matchup_recap_url",
        "status",
        "league_id",
    ],
    axis=1,
    inplace=True,
)
matchups["win_loss"] = np.where(
    matchups["winner_team_key"] == matchups["team_a_team_key"], "W", "L"
)
matchups["2pt_system"] = matchups.groupby(["week", "game_id"])["team_a_points"].rank(
    "first", ascending=False
)
matchups["2pt_system"] = np.where(matchups["2pt_system"] <= 5, 1, 0)
matchups = matchups[
    [
        "week",
        "team_a_team_key",
        "team_a_points",
        "team_a_projected_points",
        "win_loss",
        "2pt_system",
        "game_id",
    ]
]
weeks = list(matchups["week"].unique())
regular_season = matchups[matchups["week"] == 1]
for wk in weeks[1:]:
    reg_season_week = matchups[matchups["week"] == wk]
    regular_season = regular_season.merge(
        reg_season_week,
        how="outer",
        on="team_a_team_key",
        copy=True,
        suffixes=("", f"_{wk}"),
    )
reg_season_cols = list(regular_season.columns)
twopt_cols = []
team_pts = []
team_pro_pts = []
w_l_cols = []
for col in reg_season_cols:
    if "2pt_system" in col:
        twopt_cols.append(col)
    elif "team_a_points" in col:
        team_pts.append(col)
    elif "team_a_projected_points" in col:
        team_pro_pts.append(col)
    elif "win_loss" in col:
        w_l_cols.append(col)
regular_season.insert(2, "team_a_points_total", regular_season[team_pts].sum(axis=1))
regular_season.insert(
    3, "team_a_project_points_total", regular_season[team_pro_pts].sum(axis=1)
)
regular_season.insert(
    4, "wins", regular_season[w_l_cols].apply(lambda s: (s == "W").sum(), axis=1)
)
regular_season.insert(
    5, "losses", regular_season[w_l_cols].apply(lambda s: (s == "L").sum(), axis=1)
)
twopt_cols.append("wins")
regular_season.insert(2, "2pt_total", regular_season[twopt_cols].sum(axis=1))
week_1 = regular_season["week"]
regular_season.drop("week", axis=1, inplace=True)
regular_season.insert(6, "week_1", week_1)
regular_season.insert(
    2,
    "2pt_rank",
    regular_season.groupby(["game_id"])["2pt_total"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    4,
    "pts_rank",
    regular_season.groupby(["game_id"])["team_a_points_total"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    6,
    "projected_pts_rank",
    regular_season.groupby(["game_id"])["team_a_project_points_total"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    0,
    "tuple",
    regular_season[["2pt_rank", "pts_rank"]].apply(tuple, axis=1),
)
regular_season.insert(
    0,
    "reg_season_rank",
    regular_season.groupby(["game_id"])["tuple"]
    .rank(method="min", ascending=True)
    .astype(int),
)
regular_season = regular_season.merge(
    teams, how="outer", left_on="team_a_team_key", right_on="team_key"
)
team_name = regular_season["name"]
team_mang = regular_season["nickname"]
game_id = regular_season["game_id_y"]
regular_season.drop(
    ["name", "nickname", "tuple", "game_id_x", "game_id_y"],
    axis=1,
    inplace=True,
)
regular_season.insert(1, "manager", team_mang)
regular_season.insert(2, "team_name", team_name)
regular_season.insert(0, "game_id", game_id)
regular_season.sort_values(
    ["reg_season_rank", "2pt_total", "team_a_points_total"],
    ascending=[True, False, False],
    inplace=True,
)
reg_season_2pt = regular_season[
    [
        "game_id",
        "reg_season_rank",
        "manager",
        "team_name",
        "2pt_total",
        "2pt_rank",
        "team_a_points_total",
        "pts_rank",
        "team_a_project_points_total",
        "projected_pts_rank",
        "wins",
        "losses",
        "team_a_team_key",
    ]
]
DatabaseCursor(PATH, options=OPTION_PROD).copy_table_to_postgres_new(
    df=reg_season_2pt, table="regseasonrankings2pt", first_time="yes"
)

Successfully pulled: SELECT * FROM dev.weeklyleaguematchups
Successfully pulled: SELECT team_key, name, nickname, game_id FROM dev.leagueteams WHERE game_id >= 390
Upload successful: regseasonrankings2pt


#MOM FFBL Playoff Bracket

In [14]:
brack_match_query = """SELECT is_consolation, is_playoffs, week, team_a_team_key, team_a_points, team_a_projected_points, team_b_team_key, team_b_points, team_b_projected_points, game_id, status FROM dev.weeklyleaguematchups"""
brack_match = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    brack_match_query
)
reg_sea_query = """SELECT game_id, reg_season_rank, manager, team_name, team_a_team_key FROM prod.regseasonrankings2pt"""
reg_season = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    reg_sea_query
)
bracket_seeding = reg_season[reg_season["game_id"] == 406]
bracket_matchups = brack_match[
    (brack_match["game_id"] == 406) & (brack_match["is_playoffs"] == 1)
]

Successfully pulled: SELECT is_consolation, is_playoffs, week, team_a_team_key, team_a_points, team_a_projected_points, team_b_team_key, team_b_points, team_b_projected_points, game_id, status FROM dev.weeklyleaguematchups
Successfully pulled: SELECT game_id, reg_season_rank, manager, team_name, team_a_team_key FROM prod.regseasonrankings2pt


In [15]:
bracket_matchups

,is_consolation,is_playoffs,week,team_a_team_key,team_a_points,team_a_projected_points,team_b_team_key,team_b_points,team_b_projected_points,game_id,status
74,0,1,16,406.l.103661.t.1,114.98,103.87,406.l.103661.t.2,88.50,128.69,406,postevent
75,1,1,16,406.l.103661.t.3,156.88,110.97,406.l.103661.t.8,94.38,109.24,406,postevent
76,0,1,16,406.l.103661.t.4,112.92,132.40,406.l.103661.t.7,125.00,123.71,406,postevent
77,1,1,16,406.l.103661.t.6,83.86,80.28,406.l.103661.t.9,124.54,117.68,406,postevent
78,0,1,17,406.l.103661.t.1,130.70,125.13,406.l.103661.t.7,162.82,131.13,406,postevent
79,0,1,17,406.l.103661.t.2,103.94,124.62,406.l.103661.t.4,101.16,129.30,406,postevent
80,1,1,17,406.l.103661.t.3,97.46,121.63,406.l.103661.t.9,103.08,119.58,406,postevent
82,1,1,17,406.l.103661.t.6,119.30,106.31,406.l.103661.t.8,102.32,116.64,406,postevent


In [16]:
bracket_seeding

,game_id,reg_season_rank,manager,team_name,team_a_team_key
0,406,1,Tim,Fantasy Sidelines,406.l.103661.t.1
3,406,2,Ryan,King of the Hill,406.l.103661.t.4
6,406,3,Greg,Harris & Harris RBs at Law 👨🏾‍⚖️,406.l.103661.t.7
9,406,4,Wes,Dak to the Future,406.l.103661.t.2
12,406,5,Chris,Wide Out University,406.l.103661.t.6
15,406,6,Kevin,Fightin’ Blue Cock/Chubb/Wood,406.l.103661.t.3
18,406,7,Pete,Poor Decisions 😢,406.l.103661.t.8
22,406,8,Pat,Push Ups in Prison,406.l.103661.t.9
24,406,9,James,Team 10,406.l.103661.t.10
27,406,10,Jeremy,The Titanic,406.l.103661.t.5


In [46]:
def playoff_winner(team1, team2, round):

    playoff_week = bracket_matchups["week"][
        (bracket_matchups["is_playoffs"] == 1)
    ].min() + (round - 1)

    try:
        team1_score = bracket_matchups["team_a_points"][
            (bracket_matchups["team_a_team_key"] == team1)
            & (bracket_matchups["is_playoffs"] == 1)
            & (bracket_matchups["week"] == playoff_week)
        ].values[0]
    except:
        team1_score = bracket_matchups["team_b_points"][
            (bracket_matchups["team_b_team_key"] == team1)
            & (bracket_matchups["is_playoffs"] == 1)
            & (bracket_matchups["week"] == playoff_week)
        ].values[0]
    try:
        team2_score = bracket_matchups["team_a_points"][
            (bracket_matchups["team_a_team_key"] == team2)
            & (bracket_matchups["is_playoffs"] == 1)
            & (bracket_matchups["week"] == playoff_week)
        ].values[0]
    except:
        team2_score = bracket_matchups["team_b_points"][
            (bracket_matchups["team_b_team_key"] == team2)
            & (bracket_matchups["is_playoffs"] == 1)
            & (bracket_matchups["week"] == playoff_week)
        ].values[0]

    if team1_score > team2_score:
        winner = team1
    elif team1_score < team2_score:
        winner = team2
    else:
        winner = np.nan

    return winner


def games_round(games, round):
    winners = []
    for team1, team2 in games:
        winning_team = playoff_winner(team1, team2, round)
        winners.append(winning_team)

    return winners


def plan_games(teams):
    return zip(teams[::2], teams[1::2])


bracket_seeding = bracket_seeding.sort_values("reg_season_rank")
teams = list(
    bracket_seeding["team_a_team_key"][(bracket_seeding["reg_season_rank"] < 5)]
)
round = 0
while len(teams) > 1:
    round += 1
    print("Round {}: teams: {}".format(round, teams))
    games = plan_games(teams)
    teams = games_round(games, round)

champion = teams[0]  # only one left
print("Champion is {}".format(champion))

Round 1: teams: ['406.l.103661.t.1', '406.l.103661.t.4', '406.l.103661.t.7', '406.l.103661.t.2']
Round 2: teams: ['406.l.103661.t.1', '406.l.103661.t.7']
Champion is 406.l.103661.t.7


#MoM FFBL Draft Analysis/Trends Replacement

In [ ]:
"""
MoM FFBL Draft Analysis/Trends Replacement
"""
draft_query = """SELECT * FROM dev.draftresults"""
teams_query = """SELECT team_key
, game_id
, league_id
, "team_standings.outcome_totals.wins"
, "team_standings.outcome_totals.losses"
, number_of_moves
, number_of_trades
, nickname
, name
, draft_grade
, clinched_playoffs 
, "team_standings.playoff_seed"
, "team_standings.points_against" 
, "team_standings.points_for"
, "team_standings.rank"
FROM dev.leagueteams"""
players_query = """SELECT player_key
, "name.full"
, league_id
, game_id
, primary_position
, "bye_weeks.week"
, "draft_analysis.average_pick"
, "draft_analysis.average_round"
, "draft_analysis.percent_drafted"
 FROM dev.playerlist"""
draft = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(draft_query)
teams = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(teams_query)
players = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    players_query
)
draft_analysis = draft.merge(
    teams,
    how="left",
    left_on=["team_key", "game_id", "league_id"],
    right_on=["team_key", "game_id", "league_id"],
    suffixes=("_draft", "_teams"),
)
draft_analysis = draft_analysis.merge(
    players,
    how="left",
    left_on=["player_key", "game_id", "league_id"],
    right_on=["player_key", "game_id", "league_id"],
    suffixes=("", "_players"),
)
draft_analysis = draft_analysis[~draft_analysis["name"].isna()]
draft_analysis["clinched_playoffs"].fillna(0, inplace=True)
draft_analysis["team_standings.playoff_seed"].fillna(0, inplace=True)
encoded_grades = {
    "A+": 4,
    "A": 4,
    "A-": 3.7,
    "B+": 3.3,
    "B": 3.0,
    "B-": 2.7,
    "C+": 2.3,
    "C": 2.0,
    "C-": 1.7,
    "D+": 1.3,
    "D": 1.0,
    "na": np.nan,
}
draft_analysis["draft_gpa"] = draft_analysis["draft_grade"].replace(encoded_grades)
draft_analysis["avg_draft_gpa_rank"] = draft_analysis.groupby(["team_standings.rank"])[
    "draft_gpa"
].transform("mean")
draft_analysis["avg_draft_gpa_nickname"] = draft_analysis.groupby(["nickname"])[
    "draft_gpa"
].transform("mean")
draft_analysis["avg_draft_gpa_clinched_playoffs"] = draft_analysis.groupby(
    ["clinched_playoffs"]
)["draft_gpa"].transform("mean")
draft_analysis["avg_draft_gpa_playoffseed"] = draft_analysis.groupby(
    ["team_standings.playoff_seed"]
)["draft_gpa"].transform("mean")
draft_analysis["avg_draft_gpa_draft_pos"] = draft_analysis.groupby(["pick"])[
    "draft_gpa"
].transform("mean")

MoM FFBL League History Replacement

In [ ]:
"""
MoM FFBL League History Replacement
"""
matchups_query = """SELECT * FROM dev.weeklyleaguematchups"""
teams_query = """SELECT * FROM dev.leagueteams"""
draft_query = """SELECT * FROM dev.draftresults"""
players_query = """SELECT * FROM dev.playerlist"""
matchups = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    matchups_query
)
teams = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(teams_query)
draft = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(draft_query)
players = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    players_query
)
matchups = matchups.merge(
    teams,
    how="left",
    left_on=["game_id", "league_id", "team_a_team_key"],
    right_on=["game_id", "league_id", "team_key"],
    suffixes=("", "_team_a"),
)
matchups = matchups.merge(
    teams,
    how="left",
    left_on=["game_id", "league_id", "team_b_team_key"],
    right_on=["game_id", "league_id", "team_key"],
    suffixes=("_team_a", "_team_b"),
)

In [ ]:
matchups[
    (matchups["is_playoffs"] == 1) & (matchups["is_consolation"] == 0)
].sort_values(
    ["week_start", "is_consolation", "winner_team_key"], ascending=[False, True, True]
)

MoM FFBL League Records Replacement

In [ ]:
"""
MoM FFBL League Records Replacement
"""